# E-media decodowanie formatu PNG

In [ ]:
import zlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import numpy as np

In [ ]:
path = "png.png"
file = open(path, "rb")

## Weryfikacja sygnatury

#### Pierwsze 8 byte'ow pliku PNG zawiera sygnature fromatu Sygnatura PNG "137 80 78 71 13 10 26 10"

In [ ]:
def verifySignature(file):
    signature = file.read(8)
    if signature != bytes([137, 80, 78, 71, 13, 10, 26, 10]):
        print("Invalid PNG file")
        return False
    return True
verifySignature(file)

## Chunki

##### Długość

- 4-bajtowa liczba całkowita bez znaku podająca liczbę bajtów w polu danych fragmentu. Długość obejmuje tylko pole danych, a nie samo pole,

##### Typ Chunku

- Kod typu fragmentu 4-bajtowego. Dla wygody opisu i badania plików PNG, kody typu są ograniczone do wielkich i małych liter ASCII (A-Z i a-z lub 65-90 i 97-122 w systemie dziesiętnym)

##### Zawartość chunku

- Bajty danych odpowiednie dla typu fragmentu, jeśli takie istnieją. To pole może mieć długość zerową.

##### CRC
- 4-bajtowy CRC (Cyclic Redundancy Check)

In [ ]:
class Chunk:
    def __init__(self, length, type, data, crc):
        self.length = length
        self.type = type
        self.data = data
        self.crc = crc

    def __str__(self):
        return f"Chunk(type={self.type}, length={self.length})"

In [ ]:
def read_chunks(file):
    chunks = []
    while True:
        length_bytes = file.read(4)

        length = int.from_bytes(length_bytes, 'big')
        type = file.read(4).decode('utf-8')
        data = file.read(length)
        crc = file.read(4)

        chunk = Chunk(length, type, data, crc)
        chunks.append(chunk)
        print(chunk)

        if type == "IEND":
            break
    return chunks
chunks = read_chunks(file)


## Critical chunks

#### IHDR (Image Header) oraz IEND
Fragmenty mogą pojawiać się w dowolnej kolejności, z zastrzeżeniem ograniczeń nałożonych na każdy typ fragmentu. Jednym z ważniejszych ograniczeń jest to, że IHDR musi pojawić się jako pierwszy, a IEND jako ostatni

In [ ]:
def decode_IHDR(chunk):
    if chunk.type != "IHDR":
        raise ValueError("Chunk is not IHDR")
    
    image_info = {
        "width": int.from_bytes(chunk.data[0:4], 'big'),
        "height": int.from_bytes(chunk.data[4:8], 'big'),
        "bit_depth": chunk.data[8],
        "color_type": chunk.data[9],
        "compression_method": chunk.data[10],
        "filter_method": chunk.data[11],
        "interlace_method": chunk.data[12]
    }
    return image_info

IHDR = decode_IHDR(chunks[0])
print(IHDR)

### PLTE Palette

# TODO OPISAC

In [ ]:
def decode_PLTE(chunk):
    if chunk.type != "PLTE":
        raise ValueError("Not a PLTE chunk")
    
    palette = []
    for i in range(0, len(chunk.data), 3):
        r = chunk.data[i]
        g = chunk.data[i+1]
        b = chunk.data[i+2]
        palette.append((r, g, b))
    
    return palette

In [ ]:
def decode_IDAT(chunks):
    compressed_data = b''
    
    for chunk in chunks:
        if chunk.type == "IDAT":
            compressed_data += chunk.data
    try:
        decompressed_data = zlib.decompress(compressed_data)
        return decompressed_data
    except zlib.error as e:
        return None
    
def decode_scanlines(decompressed_data, width, height):
    scanlines = []
    byte_index = 0
    for _ in range(height):
        # Separator byte for each scanline
        byte_index += 1
        
        row_pixels = []
        for _ in range(width):
            r = decompressed_data[byte_index]
            g = decompressed_data[byte_index + 1]
            b = decompressed_data[byte_index + 2]
            row_pixels.append((r, g, b))
            byte_index += 3
        scanlines.append(row_pixels)
    
    return scanlines

decompressed_data = decode_IDAT(chunks)
decompressed_data = decode_scanlines(decompressed_data, IHDR["width"] , IHDR["height"])

## Ancillary chunks

- Wszystkie dodatkowe fragmenty są opcjonalne,
- Fragmenty opcjonalne zaczynaja sie od małych liter

In [ ]:
def decode_Ancillary(chunks):
    for chunk in chunks: 
        if chunk.type in ["tEXt", "iTXt"]:
            try:
                decoded_text = chunk.data.decode('utf-8')
                print(f">> {decoded_text}")
            except Exception as e:
                decoded_text = chunk.data.decode('latin-1')
                print(f">> {decoded_text}")

        if chunk.type == "zTXt":
            null_index = chunk.data.index(b'\x00')
            keyword = chunk.data[:null_index].decode('utf-8')
            compression = chunk.data[null_index + 1]
            compressed_text = chunk.data[null_index + 2:]
            if compression == 0:
                decompressed_text = zlib.decompress(compressed_text).decode('utf-8')
                print(f">> {decompressed_text}")

        #if chunk.type == "bKGD":
        # TODO
        #if chunk.type == "gAMA":
        # TODO
        #if chunk.type == "pHYs":
        # TODO
        #if chunk.type == "tIME":
decode_Ancillary(chunks)

In [ ]:
file.close()

## Anonymize

### Przy anonimizacji, czyścić dodatkowe segmenty (ang. ancillary chunks).

In [ ]:
def anonymize_clear_png(input_path, output_path):

    critical_chunks = {b'IHDR', b'PLTE', b'IDAT', b'IEND'}
    
    with open(input_path, 'rb') as infile, open(output_path, 'wb') as outfile:
        outfile.write(infile.read(8))
        
        while True:
            length_bytes = infile.read(4)
            if not length_bytes:
                break
                
            length = int.from_bytes(length_bytes, 'big')
            chunk_type = infile.read(4)
            chunk_data = infile.read(length)
            crc = infile.read(4)

            if chunk_type in critical_chunks:
                outfile.write(length_bytes)
                outfile.write(chunk_type)
                outfile.write(chunk_data)
                outfile.write(crc)
            
            if chunk_type == b'IEND':
                break

### Przy anonimizacji, uwzględnić możliwość zapisania danych w sposobie konstrukcji pliku

In [ ]:
def anonymize_save_png(input_path, output_path):
    critical_chunks = {b'IHDR', b'PLTE', b'IDAT', b'IEND'}
    
    with open(input_path, 'rb') as infile, open(output_path, 'wb') as outfile:
        outfile.write(infile.read(8))\
    
    # TODO

In [ ]:
def anonymize_png(input_path, output_path, option=True):
    if option:
        anonymize_clear_png(input_path, output_path)
    else:
        anonymize_save_png(input_path, output_path)

In [ ]:
anonymize_clear_png("png.png", "anonymized_png.png")

with open("png.png", "rb") as file:
    file.seek(8)
    print(read_chunks(file))

with open("anonymized_png.png", "rb") as file:
    file.seek(8)
    print(read_chunks(file))

## Transformacja

In [ ]:
def showSpectrum(path, palette=None):
    """Display frequency spectrum for PNG images based on color type"""
    orig_img = Image.open(path)
    color_type = IHDR["color_type"]
    
    plt.figure(figsize=(15, 8))
    
    # Grayscale (color type 0)
    if color_type == 0:
        grayscale_img = orig_img.convert('L')
        mag_spectrum, phase_spectrum = getMagnitudeSpectrum(np.array(grayscale_img))
        
        plt.subplot(1, 2, 1)
        plt.imshow(mag_spectrum, cmap='gray')
        plt.title("Magnitude Spectrum")
        
        plt.subplot(1, 2, 2)
        plt.imshow(phase_spectrum, cmap='gray')
        plt.title("Phase Spectrum")
    
    # Truecolor (color type 2)
    elif color_type == 2:
        img_array = np.array(orig_img)
        
        for i, (color, channel) in enumerate(zip(['Red', 'Green', 'Blue'], np.rollaxis(img_array, -1))):
            mag, phase = getMagnitudeSpectrum(channel)
            
            plt.subplot(2, 3, i+1)
            plt.imshow(mag, cmap='gray')
            plt.title(f"{color} Channel Magnitude")
            
            plt.subplot(2, 3, i+4)
            plt.imshow(phase, cmap='gray')
            plt.title(f"{color} Channel Phase")
    
    # Indexed color (color type 3)
    elif color_type == 3 and palette is not None:
        img_array = np.array(orig_img)
        palette_rgb = np.array(palette, dtype=np.uint8).reshape(-1, 3)
        rgb_array = palette_rgb[img_array]
        
        for i, (color, channel) in enumerate(zip(['Red', 'Green', 'Blue'], np.rollaxis(rgb_array, -1))):
            mag, phase = getMagnitudeSpectrum(channel)
            
            plt.subplot(2, 3, i+1)
            plt.imshow(mag, cmap='gray')
            plt.title(f"{color} Channel Magnitude")
            
            plt.subplot(2, 3, i+4)
            plt.imshow(phase, cmap='gray')
            plt.title(f"{color} Channel Phase")
    
    # Grayscale with alpha (color type 4)
    elif color_type == 4:
        img_array = np.array(orig_img)
        grayscale = img_array[:, :, 0]  # Use only the grayscale channel
        
        mag, phase = getMagnitudeSpectrum(grayscale)
        
        plt.subplot(1, 2, 1)
        plt.imshow(mag, cmap='gray')
        plt.title("Grayscale Magnitude Spectrum")
        
        plt.subplot(1, 2, 2)
        plt.imshow(phase, cmap='gray')
        plt.title("Grayscale Phase Spectrum")
    
    # Truecolor with alpha (color type 6)
    elif color_type == 6:
        img_array = np.array(orig_img)
        
        for i, (color, channel) in enumerate(zip(['Red', 'Green', 'Blue'], np.rollaxis(img_array[:, :, :3], -1))):
            mag, phase = getMagnitudeSpectrum(channel)
            
            plt.subplot(2, 3, i+1)
            plt.imshow(mag, cmap='gray')
            plt.title(f"{color} Channel Magnitude")
            
            plt.subplot(2, 3, i+4)
            plt.imshow(phase, cmap='gray')
            plt.title(f"{color} Channel Phase")
    
    plt.tight_layout()
    plt.show()

def getMagnitudeSpectrum(image_data):
    """Calculate magnitude and phase spectrum of an image"""
    f = np.fft.fft2(image_data)
    fshift = np.fft.fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1e-10)
    phase = np.angle(fshift)
    return magnitude, phase

showSpectrum("png.png")

## Źródła

- http://www.libpng.org/pub/png/spec/1.2/PNG-Structure.html